2025.05.12, Zhang Chao

In [1]:
import pandas as pd
import xarray as xr
import regionmask
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('/home/climate/chaoz/code/utils/')
from plot_utils import plot_settings

In [2]:
os.chdir('/home/climate/chaoz/project/03Irr_Ts_CN/processed/')
dLE_base = xr.open_dataset('delta_LE_PMLv2_Yr_CN_2015_IrrMap_CN.nc').squeeze(dim='time')
dLE_meier = xr.open_dataset('delta_LE_PMLv2_Yr_CN_2015_IrrmapMeier.nc').squeeze(dim='time')
dLST_base = xr.open_dataset('delta_LSTday_Yr_CN_2001_2020.nc').sel(time=slice('2015','2015')).squeeze(dim='time')
dLST_meier = xr.open_dataset('delta_LSTday_Yr_CN_2015_IrrmapMeier.nc').squeeze(dim='time')

shp_climzone = gpd.read_file('../shapefile/ClimateZone_3.shp')

In [3]:
def stats_regionmean_std(ds, varname, shp):
    # Ensure we are working with the specific variable DataArray
    da = ds[varname]

    # --- National Calculation ---
    # Count non-null values for the specific variable nationally
    national_count = da.notnull().sum().item()
    # Calculate national mean
    cn_mean = da.mean(dim=('lat', 'lon')).item()
    # Calculate national standard deviation
    cn_std_dev = da.std(dim=('lat', 'lon')).item()

    # --- Regional Calculation ---
    # Create the region mask
    mask_region = regionmask.mask_geopandas(shp, da.lon, da.lat)

    # Calculate regional means
    region_mean = da.groupby(mask_region).mean()
    # Calculate regional standard deviations
    region_std_dev = da.groupby(mask_region).std()

    # Convert regional results to DataFrames
    region_mean_df = region_mean.to_dataframe(name='mean').reset_index()
    # Use the calculated regional_std directly
    region_std_df = region_std_dev.to_dataframe(name='std').reset_index()

    if 'name' in shp.columns: # Example: if shp has a 'name' column
         mask_mapping = {i: name for i, name in enumerate(shp['name'])}
    else: # Fallback to your original mapping if no name column
        mask_mapping = {1.0:'Arid', 2.0:'Semi', 0.0:'Humid'}
        print("Warning: Using default mask mapping. Verify it matches your shapefile regions.")


    region_mean_df['mask'] = region_mean_df['mask'].replace(mask_mapping)
    region_std_df['mask'] = region_std_df['mask'].replace(mask_mapping)

    # --- Combine Results ---
    # Merge regional mean and std based on the mask name
    df1 = pd.merge(region_mean_df[['mask', 'mean']],
                   region_std_df[['mask', 'std']],
                   on='mask')

    # Create DataFrame for national results
    df2 = pd.DataFrame({'mask': ['China'], # Assuming 'China' is the national label
                        'mean': [cn_mean],
                        'std': [cn_std_dev]})

    # Concatenate regional and national results
    df3 = pd.concat([df1, df2], axis=0, ignore_index=True).set_index('mask')

    # Reindex to desired order (ensure region names match those from mask_mapping)
    # Get the region names from the mapping used
    region_order = ['China'] + list(mask_mapping.values())
    # Filter df3 index to only include expected regions before reindexing
    df3 = df3[df3.index.isin(region_order)]
    df3 = df3.reindex(region_order) # Use the actual region names

    return df3


In [ ]:
df_LSTbase  = stats_regionmean_std(dLST_base,'Ts',shp_climzone)
df_LSTmeier = stats_regionmean_std(dLST_meier,'Ts',shp_climzone)
df_LEbase   = stats_regionmean_std(dLE_base,'LE',shp_climzone)
df_LEmeier  = stats_regionmean_std(dLE_meier,'LE',shp_climzone)

In [5]:
def plot_bars(ax,mean,std,no,ylim,ylabel):
    ax.set_ylim(ylim)
    ax.axhline(0, 0, 10, color='k',linestyle='--',linewidth=0.8)
    colors = ['gray','#FFA726','#33A02C','#1E88E5']
    err_attri = dict(elinewidth=.8, ecolor='k', capsize=4,alpha=0.5)
    ax.bar(range(1,len(mean)+1),mean,yerr=std,color=colors,error_kw=err_attri)
    ax.text(-0.05,1.05,no,weight='bold',transform=ax.transAxes,fontsize=14)
    ax.set_ylabel(ylabel)
    ax.set_xticks([1,2,3,4],['China','Arid','Semi','Humid'])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    return ax

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(9,6),sharex=False)
fig.subplots_adjust(left = 0.07, right = 0.99,
                    bottom = 0.05, top = 0.96,
                    wspace= 0.12, hspace=0.25)
plot_settings()
ylim1,ylim2 = [-5,1.1],[-0.19,0.79]
ylabel1,ylabel2 = '$\Delta$LST_Day [K]','$\Delta$ET [mm/day]'
plot_bars(axes[0,0],df_LSTbase['mean'] ,df_LSTbase['std'] ,'a',ylim1,ylabel1)
plot_bars(axes[0,1],df_LSTmeier['mean'],df_LSTmeier['std'],'b',ylim1,'')
plot_bars(axes[1,0],df_LEbase['mean']  ,df_LEbase['std']  ,'c',ylim2,ylabel2)
plot_bars(axes[1,1],df_LEmeier['mean'] ,df_LEmeier['std'] ,'d',ylim2,'')

plt.savefig('../figures/Figure_S16.png',dpi=300)